In [1]:
#! /usr/bin/env python3
import multiCMD
import TSVZ
import inotify_simple
import Tee_Logger
import argparse
import xxhash

from collections import deque
import time
from collections import namedtuple
from collections import OrderedDict
import os
import threading
import sys
import signal
import resource
import datetime
import unicodedata
import re
from shutil import copystat

# for the lib wrapping aroung inotify, I tried
# wtr-wather, watchdog, watchfiles
# None of them seem to work well with the most basic tests
# I need it to work with:
# 1. able to notice changes recursively
# 2. able to notice changes in the folders
# 3. do not incorrectly group move events
# 4. recognize mkdir -p properly
# 5. notice umount events and notify properly
# 6. notice mount events and notify properly
# 7. compatible with symlink events 


__version__ = 0.01

## WIP

# @dataclass
# class ChangeEvent:
# 	monotonic_time:float
# 	# type can be file or dir.
# 	type:str
# 	# if type is dir, event can be create 
# 	#  ( need to recursively copy everything as inotifywait does not caputre mkdir -p properly for all sub folders),
# 	#  ,delete, attrib, move
# 	# if type is file, event can be delete, change, attrib, move
# 	# move detection will be handled by monitor if moved_from moved_to events were captured. 
# 	# it will use an tsv storing all the path inode number and hash to detect move.
# 	event:str
# 	path:str
# 	moved_from:str = None


# if type is dir, event can be create 
#  ( need to recursively copy everything as inotifywait does not caputre mkdir -p properly for all sub folders),
#  ,delete, attrib, move
# if type is file, event can be delete, change, attrib, move
# move detection will be handled by monitor if moved_from moved_to events were captured. 
# it will use an tsv storing all the path inode number and hash to detect move.
CHANGED_EVENT_HEADER = ['monotonic_time', 'is_dir', 'event', 'path','moved_from']
BACKUP_ENTRY_VALUES_HEADER = ['iso_time','event','source_path']
BACKUP_JOURNAL_HEADER = ['at_path','iso_time','event','source_path']
REVERB_RULE_TSV_HEADER = ['Job Name (Unique Key)', 'Path Monitoring', 'Monitoring File System Signiture', 
		'Minium snapshot time', 'Vault Path', 'Keep 1 Compelete Backup', 
		'Only Sync Attributes (permissions)', 'Keep N versions', 'Backup Size Limit', 
		'Vault File System Signiture']
REVERB_RULE_HEADER = ['job_name', 'mon_path', 'mon_fs_signiture', 
		'min_shapshot_time', 'vault_path', 'keep_one_complete_backup', 
		'only_sync_attributes', 'keep_n_versions', 'backup_size_limit',
		'vault_fs_signiture']
VAULT_ENTRY_HEADER = ['version_number','path','timestamp','size','inode']
VAULT_TIMESTAMP_FORMAT = '%Y-%m-%d_%H-%M-%S_%z'

ChangeEvent = namedtuple('ChangeEvent', CHANGED_EVENT_HEADER)
BackupEntryValues = namedtuple('BackupEntryValues', BACKUP_ENTRY_VALUES_HEADER)
ReverbRule = namedtuple('ReverbRule', REVERB_RULE_HEADER)
VaultEntry = namedtuple('VaultEntry', VAULT_ENTRY_HEADER)

# by defualt, set the max backup threads to 2* the number of cores
BACKUP_SEMAPHORE = threading.Semaphore(2*os.cpu_count())

DEFAULT_SNAPSHOT_DELAY = 60
DEFAULT_KEEP_ONE_COMPLETE_BACKUP = False
DEFAULT_ONLY_SYNC_ATTRIBUTES = False
DEFAULT_KEEP_N_VERSIONS = 30
DEFAULT_BACKUP_SIZE_LIMIT = '5%'

ARG_MAX = os.sysconf('SC_ARG_MAX')
ARGUMENT_LIMIT = (ARG_MAX - 4096) // 2048

COOKIE_DICT_MAX_SIZE = 16384
COOKIE_VALUE = namedtuple('COOKIE_VALUE',['wd','path','isDir'])

GREEN_LIGHT = threading.Event()
GREEN_LIGHT.set()

DEBUG = True

tl = Tee_Logger.teeLogger(systemLogFileDir='/dev/null', programName='reverberator', compressLogAfterMonths=0, deleteLogAfterYears=0, suppressPrintout=False,noLog=True)


In [140]:
def get_path_size(*path:str):
	global DEBUG
	# this function gets the actual size of a path
	# du --bytes -s <path>
	if DEBUG:
		startTime = time.perf_counter()
	rtn = multiCMD.run_command(['du','--block-size=1','-csP',*path],quiet=True)
	if DEBUG:
		print(f'get_path_size took {time.perf_counter()-startTime} seconds')
	if rtn and rtn[-1] and rtn[-1].partition('\t')[0].isdigit():
		return int(rtn[-1].partition('\t')[0])
	else:
		return 0

In [2]:
def format_bytes(size, use_1024_bytes=None, to_int=False, to_str=False,str_format='.2f'):
	"""
	Format the size in bytes to a human-readable format or vice versa.
	From hpcp: https://github.com/yufei-pan/hpcp

	Args:
		size (int or str): The size in bytes or a string representation of the size.
		use_1024_bytes (bool, optional): Whether to use 1024 bytes as the base for conversion. If None, it will be determined automatically. Default is None.
		to_int (bool, optional): Whether to convert the size to an integer. Default is False.
		to_str (bool, optional): Whether to convert the size to a string representation. Default is False.
		str_format (str, optional): The format string to use when converting the size to a string. Default is '.2f'.

	Returns:
		int or str: The formatted size based on the provided arguments.

	Examples:
		>>> format_bytes(1500)
		'1.50 KB'
		>>> format_bytes('1.5 GiB', to_int=True)
		1610612736
	"""
	if to_int or isinstance(size, str):
		if isinstance(size, int):
			return size
		elif isinstance(size, str):
			# Use regular expression to split the numeric part from the unit, handling optional whitespace
			match = re.match(r"(\d+(\.\d+)?)\s*([a-zA-Z]*)", size)
			if not match:
				print("Invalid size format. Expected format: 'number [unit]', e.g., '1.5 GiB' or '1.5GiB'")
				print(f"Got: {size}")
				return 0
			number, _, unit = match.groups()
			number = float(number)
			unit  = unit.strip().lower().rstrip('b')
			# Define the unit conversion dictionary
			if unit.endswith('i'):
				# this means we treat the unit as 1024 bytes if it ends with 'i'
				use_1024_bytes = True
			elif use_1024_bytes is None:
				use_1024_bytes = False
			unit  = unit.rstrip('i')
			if use_1024_bytes:
				power = 2**10
			else:
				power = 10**3
			unit_labels = {'': 0, 'k': 1, 'm': 2, 'g': 3, 't': 4, 'p': 5}
			if unit not in unit_labels:
				print(f"Invalid unit '{unit}'. Expected one of {list(unit_labels.keys())}")
				return 0
			# Calculate the bytes
			return int(number * (power ** unit_labels[unit]))
		else:
			try:
				return int(size)
			except Exception as e:
				return 0
	elif to_str or isinstance(size, int) or isinstance(size, float):
		if isinstance(size, str):
			try:
				size = size.lower().strip().rstrip('b')
				size = float(size)
			except Exception as e:
				return size
		# size is in bytes
		if use_1024_bytes or use_1024_bytes is None:
			power = 2**10
			n = 0
			power_labels = {0 : '', 1: 'Ki', 2: 'Mi', 3: 'Gi', 4: 'Ti', 5: 'Pi'}
			while size > power:
				size /= power
				n += 1
			return f"{size:{str_format}}{' '}{power_labels[n]}"
		else:
			power = 10**3
			n = 0
			power_labels = {0 : '', 1: 'K', 2: 'M', 3: 'G', 4: 'T', 5: 'P'}
			while size > power:
				size /= power
				n += 1
			return f"{size:{str_format}}{' '}{power_labels[n]}"
	else:
		try:
			return format_bytes(float(size), use_1024_bytes)
		except Exception as e:
			import traceback
			print(f"Error: {e}")
			print(traceback.format_exc())
			print(f"Invalid size: {size}")
		return 0


In [52]:
def get_path_inodes(*path:str):
	global DEBUG
	# this function gets the number of inodes in a path
	# df --inodes -s <path>
	if DEBUG:
		startTime = time.perf_counter()
	rtn = multiCMD.run_command(['du','--inodes','-cs',*path],quiet=True)
	if DEBUG:
		print(f'get_path_inodes of {[*path]} took {time.perf_counter()-startTime} seconds')
	if rtn and rtn[-1] and rtn[-1].partition('\t')[0].isdigit():
		return int(rtn[-1].partition('\t')[0])
	else:
		return 0


In [138]:
def get_vault_info(job_vault_path:str,recalculate:bool=False) -> tuple:
	# job vault subfolder should follow: V{version}--{ISO8601ish time}--{size of this backup}-{inodes of this backup}
	# ex. V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino
	global DEBUG
	global tl
	global VAULT_TIMESTAMP_FORMAT
	if not os.path.exists(job_vault_path):
		tl.teeerror(f'Job vault path {job_vault_path} does not exist')
		return {}, 0, 0
	# if the symlink is broken, we will try to find the latest version folder
	vault_info_dict = {}
	vault_size = 0
	vault_inodes = 0
	try:
		for entry in os.scandir(job_vault_path):
			if entry.is_dir() and entry.name.startswith('V') and '--' in entry.name:
				version_number_str = entry.name.lstrip('V').partition('--')[0]
				if version_number_str.isdigit():
					version_number = int(version_number_str)
					try:
						if not recalculate:
							entry_size_inode_str = entry.name.rpartition('--')[2]
							entry_size_str = entry_size_inode_str.partition('-')[0].replace('_','')
							entry_size = format_bytes(entry_size_str,to_int=True)
							entry_inode_str = entry_size_inode_str.rpartition('-')[2].replace('_ino','')
							entry_inode = format_bytes(entry_inode_str,to_int=True)
							vault_size += entry_size
							vault_inodes += entry_inode
							entry_timestamp = datetime.datetime.strptime(entry.name.partition('--')[2].rpartition('--')[0],VAULT_TIMESTAMP_FORMAT).timestamp()
							#VAULT_ENTRY_HEADER = ['version_number','path','timestamp','size','inode']
							vault_info_dict[version_number] = VaultEntry(version_number,entry.path,entry_timestamp,entry_size,entry_inode)
						else:
							# we will recalculate the size and inodes for the folders and rename the folders accordingly
							entry_size = get_path_size(entry.path)
							entry_inode = get_path_inodes(entry.path)
							vault_size += entry_size
							vault_inodes += entry_inode
							backup_size_str = format_bytes(entry_size,use_1024_bytes=True,to_str=True).replace(' ','_')
							backup_inode_str = format_bytes(entry_inode,use_1024_bytes=False,to_str=True).replace(' ','')
							entry_timestamp = datetime.datetime.strptime(entry.name.partition('--')[2].rpartition('--')[0],VAULT_TIMESTAMP_FORMAT).timestamp()
							new_entry_name = f'V{version_number}--{datetime.datetime.fromtimestamp(entry_timestamp).astimezone().strftime(VAULT_TIMESTAMP_FORMAT)}--{backup_size_str}B-{backup_inode_str}_ino'
							if entry.name != new_entry_name:
								tl.teeprint(f'Renaming {entry.name} to {new_entry_name}')
								try:
									new_entry_path = os.path.join(job_vault_path,new_entry_name)
									os.rename(entry.path,new_entry_path)
									vault_info_dict[version_number] = VaultEntry(version_number,new_entry_path,entry_timestamp,entry_size,entry_inode)
								except Exception as e:
									tl.teeerror(f'Error renaming {entry.name} to {new_entry_name}: {e}')
									continue
							else:
								vault_info_dict[version_number] = VaultEntry(version_number,entry.path,entry_timestamp,entry_size,entry_inode)
					except:
						if DEBUG:
							import traceback
							tl.teeerror(f'Error processing {entry.name}: {traceback.format_exc()}')
	except PermissionError:
		tl.teeerror(f'Permission error while scanning {job_vault_path}')
	return OrderedDict(sorted(vault_info_dict.items())), vault_size, vault_inodes


In [47]:
format_bytes(1, to_str=True)

'1.00 '

In [141]:
get_vault_info('test_vault',recalculate=True)

get_path_size took 0.004089745998498984 seconds
get_path_inodes of ['test_vault/V3--2021-01-01_00-00-00_-0800--2.17_KiB-100.00_ino'] took 0.003859294010908343 seconds
Renaming V3--2021-01-01_00-00-00_-0800--2.17_KiB-100.00_ino to V3--2021-01-01_00-00-00_-0800--132.00_KiB-100.00_ino
get_path_size took 0.005838608005433343 seconds
get_path_inodes of ['test_vault/V4--2021-12-31_16-00-00_-0800--2.76_KiB-109.00_ino'] took 0.0039017529925331473 seconds
Renaming V4--2021-12-31_16-00-00_-0800--2.76_KiB-109.00_ino to V4--2021-12-31_16-00-00_-0800--164.00_KiB-109.00_ino
get_path_size took 0.0018159080063924193 seconds
get_path_inodes of ['test_vault/V2--2020-01-01_00-00-00_-0800--56.00_B-2.00_ino'] took 0.0013366789935389534 seconds
Renaming V2--2020-01-01_00-00-00_-0800--56.00_B-2.00_ino to V2--2020-01-01_00-00-00_-0800--4.00_KiB-2.00_ino
get_path_size took 0.0015047140041133389 seconds
get_path_inodes of ['test_vault/V0--2021-01-01_00-00-00_-0800--13.00_B-3.00_ino'] took 0.00123979699856136 se

(OrderedDict([(0,
               VaultEntry(version_number=0, path='test_vault/V0--2021-01-01_00-00-00_-0800--8.00_KiB-3.00_ino', timestamp=1609488000.0, size=8192, inode=3)),
              (1,
               VaultEntry(version_number=1, path='test_vault/V1--2023-01-01_00-00-00_-0800--4.00_KiB-2.00_ino', timestamp=1672560000.0, size=4096, inode=2)),
              (2,
               VaultEntry(version_number=2, path='test_vault/V2--2020-01-01_00-00-00_-0800--4.00_KiB-2.00_ino', timestamp=1577865600.0, size=4096, inode=2)),
              (3,
               VaultEntry(version_number=3, path='test_vault/V3--2021-01-01_00-00-00_-0800--132.00_KiB-100.00_ino', timestamp=1609488000.0, size=135168, inode=100)),
              (4,
               VaultEntry(version_number=4, path='test_vault/V4--2021-12-31_16-00-00_-0800--164.00_KiB-109.00_ino', timestamp=1640995200.0, size=167936, inode=109))]),
 319488,
 216)

In [96]:
vault_info_dict, vault_size , vault_inodes= get_vault_info('test_vault')

In [97]:
vault_info_dict[next(reversed(vault_info_dict))]

VaultEntry(version_number=4, path='test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino', timestamp=1640995200.0, size=1153433, inode=4)

In [51]:
get_path_size('ideas.txt','uv.lock')

get_path_size of ['ideas.txt', 'uv.lock'] took 0.004271733996574767 seconds


45242

In [53]:
get_path_inodes(os.path.expanduser('~'))

get_path_inodes of ['/var/home/kes'] took 1.0356939350022003 seconds


187792

In [54]:
get_path_inodes('ideas.txt','uv.lock')

get_path_inodes of ['ideas.txt', 'uv.lock'] took 0.0021252570004435256 seconds


2

In [55]:
def run_commands_with_semaphore(commands:list):
	global BACKUP_SEMAPHORE
	global DEBUG
	global tl
	# check if there is any free semaphores available
	# Use as much semaphore as possible
	remaining_semaphores = BACKUP_SEMAPHORE._value
	if remaining_semaphores < len(commands) // 2:
		threads_to_use = remaining_semaphores // 2
	else:
		threads_to_use = len(commands) // 2
	if DEBUG:
		tl.teeprint(f'Using {threads_to_use} threads for backup')
	if threads_to_use > 1:
		[BACKUP_SEMAPHORE.acquire() for _ in range(threads_to_use)]
		if DEBUG:
			tl.teeprint(f'Running backup commands with {threads_to_use} threads, reduced sem from {remaining_semaphores} to {BACKUP_SEMAPHORE._value}')
		multiCMD.run_commands(commands,quiet=True,return_code_only=True,max_threads=threads_to_use,wait_for_return=True)
		BACKUP_SEMAPHORE.release(threads_to_use)
		if DEBUG:
			tl.teeprint(f'Released {threads_to_use} semaphores, sem count: {BACKUP_SEMAPHORE._value}')
	else:
		multiCMD.run_commands(commands,quiet=True,return_code_only=True,wait_for_return=True)

In [56]:
commands = ['echo "hello world"']*10
run_commands_with_semaphore(commands=commands)

Using 5 threads for backup
Running backup commands with 5 threads, reduced sem from 32 to 27
Released 5 semaphores, sem count: 32


In [57]:
#get running threads
for thread in threading.enumerate():
    print(f"Thread Name: {thread.name}, Alive: {thread.is_alive()}")

Thread Name: MainThread, Alive: True
Thread Name: IOPub, Alive: True
Thread Name: Heartbeat, Alive: True
Thread Name: Thread-1 (_watch_pipe_fd), Alive: True
Thread Name: Thread-2 (_watch_pipe_fd), Alive: True
Thread Name: Control, Alive: True
Thread Name: IPythonHistorySavingThread, Alive: True


In [58]:
import json
loaded_entries = json.loads('''
{
  "/tmp/inotify_test/write_test.txt": [
    "2025-03-25T18:11:41.157841",
    "delete",
    null
  ],
  "/tmp/inotify_test/same_file_move_out_in.txt": [
    "2025-03-25T18:11:41.157841",
    "create",
    null
  ],
  "/tmp/inotify_test/dirMoveTest/": [
    "2025-03-25T18:11:41.157841",
    "move",
    "/tmp/inotify_test/newDirTest/"
  ],
  "/tmp/inotify_test/dirMoveTest/subfolder.txt": [
    "2025-03-25T18:11:41.157841",
    "create",
    null
  ],
  "/tmp/inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt": [
    "2025-03-25T18:11:41.157841",
    "create",
    "/tmp/inotify_test/dirMoveTest/write_to_sub_folder.txt"
  ],
  "/tmp/inotify_test/dirMoveTest/move_into_sub_folder.txt": [
    "2025-03-25T18:11:41.157841",
    "create",
    "/tmp/inotify_test/new_file.txt"
  ],
  "/tmp/inotify_test/dirMoveTest/test_temp.txt": [
    "2025-03-25T18:11:41.157841",
    "create",
    null
  ]
}
''')
backup_entries= OrderedDict()
for entry in loaded_entries:
	backup_entries[entry] = BackupEntryValues(*loaded_entries[entry])
backup_entries

OrderedDict([('/tmp/inotify_test/write_test.txt',
              BackupEntryValues(iso_time='2025-03-25T18:11:41.157841', event='delete', source_path=None)),
             ('/tmp/inotify_test/same_file_move_out_in.txt',
              BackupEntryValues(iso_time='2025-03-25T18:11:41.157841', event='create', source_path=None)),
             ('/tmp/inotify_test/dirMoveTest/',
              BackupEntryValues(iso_time='2025-03-25T18:11:41.157841', event='move', source_path='/tmp/inotify_test/newDirTest/')),
             ('/tmp/inotify_test/dirMoveTest/subfolder.txt',
              BackupEntryValues(iso_time='2025-03-25T18:11:41.157841', event='create', source_path=None)),
             ('/tmp/inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt',
              BackupEntryValues(iso_time='2025-03-25T18:11:41.157841', event='create', source_path='/tmp/inotify_test/dirMoveTest/write_to_sub_folder.txt')),
             ('/tmp/inotify_test/dirMoveTest/move_into_sub_folder.txt',
         

In [59]:
def do_complete_backup(source_path:str,backup_folder:str):
	global DEBUG
	multiCMD.run_command(['cp','-a','--reflink=auto','--sparse=always',source_path,backup_folder],quiet=True,return_code_only=True)


In [60]:
do_complete_backup('/tmp','test')

In [61]:
def get_all_files_and_folders(path):
	files = []
	folders = []
	try:
		with os.scandir(path) as entries:
			for entry in entries:
				if entry.is_dir(follow_symlinks=False):
					folders.append(entry.path+'/')
					child_files, child_folders = get_all_files_and_folders(entry.path)
					files.extend(child_files)
					folders.extend(child_folders)
				else:
					files.append(entry.path)
	except PermissionError:
		pass
	return files, folders


In [62]:
def do_referenced_copy(source_path:str,backup_folder:str):
	global DEBUG
	global tl
	global BACKUP_SEMAPHORE
	files, folders = get_all_files_and_folders(source_path)
	for folder in folders:
		try:
			rel_folder = os.path.relpath(folder,source_path)
			backup_folder_path = os.path.join(backup_folder,rel_folder)
			os.makedirs(backup_folder_path,exist_ok=True)
			# copy the folder metadata
			copystat(folder,backup_folder_path)
			st = os.stat(folder)
			if os.name == 'posix':
				os.chown(backup_folder_path, st.st_uid, st.st_gid)
			os.utime(backup_folder_path, (st.st_atime, st.st_mtime))
		except:
			tl.teeerror(f'Failed to copy folder {folder}')
	commands = []
	for file in files:
		# use ln -fsrLT to do a relative symlink
		# ln --symbolic --logical --force --no-target-directory
		rel_file = os.path.relpath(file,source_path)
		backup_file_path = os.path.join(backup_folder,rel_file)
		file = os.path.abspath(file)
		commands.append(['ln','-fsLT',file,backup_file_path])
	if commands:
		run_commands_with_semaphore(commands)


In [63]:
do_referenced_copy('test','test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino')

Using 20 threads for backup
Running backup commands with 20 threads, reduced sem from 32 to 12
Released 20 semaphores, sem count: 32


In [64]:
def get_backup_size_inode(backup_entries:dict,only_sync_attributes:bool): 
	'''
	This function gets the size of the backup entries

	Parameters:
		backup_entries (dict): The backup entries
		only_sync_attributes (bool): Whether to only sync the attributes

	Returns:
		tuple: The size and inode change compared to previous backup of the backup entries
	'''
	# BACKUP_ENTRY_VALUES_HEADER = ['iso_time','event','source_path']
	# event can be create, modify, attrib, move, delete
	total_size = 0
	total_inode_change = 0
	path_pending_to_get_size = []
	path_pending_to_get_inode_add = []
	path_pending_to_get_inode_del = []
	for entry in backup_entries:
		if backup_entries[entry].event == 'create' :
			path_pending_to_get_size.append(entry)
			path_pending_to_get_inode_add.append(entry)
		elif backup_entries[entry].event == 'modify' or (not only_sync_attributes and backup_entries[entry].event == 'attrib'):
			path_pending_to_get_size.append(entry)
			# we do not count inode change for attrib / modify / move events
		elif backup_entries[entry].event == 'delete':
			path_pending_to_get_inode_del.append(entry)
			# size will not decrease as we are not backing up the full content, but inodes will decrease as we are using symlinks
		if len(path_pending_to_get_size) > ARGUMENT_LIMIT:
			total_size += get_path_size(*path_pending_to_get_size)
			path_pending_to_get_size = []
		if len(path_pending_to_get_inode_add) > ARGUMENT_LIMIT:
			total_inode_change += get_path_inodes(*path_pending_to_get_inode_add)
			path_pending_to_get_inode_add = []
		if len(path_pending_to_get_inode_del) > ARGUMENT_LIMIT:
			total_inode_change -= get_path_inodes(*path_pending_to_get_inode_del)
			path_pending_to_get_inode_del = []
	if path_pending_to_get_size:
		total_size += get_path_size(*path_pending_to_get_size)
	if path_pending_to_get_inode_add:
		total_inode_change += get_path_inodes(*path_pending_to_get_inode_add)
	if path_pending_to_get_inode_del:
		total_inode_change -= get_path_inodes(*path_pending_to_get_inode_del)
	return total_size, total_inode_change


In [65]:
get_backup_size_inode(backup_entries,only_sync_attributes=True)

get_path_size of ['/tmp/inotify_test/same_file_move_out_in.txt', '/tmp/inotify_test/dirMoveTest/subfolder.txt', '/tmp/inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt', '/tmp/inotify_test/dirMoveTest/move_into_sub_folder.txt', '/tmp/inotify_test/dirMoveTest/test_temp.txt'] took 0.003279122000094503 seconds
get_path_inodes of ['/tmp/inotify_test/same_file_move_out_in.txt', '/tmp/inotify_test/dirMoveTest/subfolder.txt', '/tmp/inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt', '/tmp/inotify_test/dirMoveTest/move_into_sub_folder.txt', '/tmp/inotify_test/dirMoveTest/test_temp.txt'] took 0.0023942620027810335 seconds
get_path_inodes of ['/tmp/inotify_test/write_test.txt'] took 0.0008840579976094887 seconds


(0, 0)

In [66]:
def get_backup_limits_from_str(backup_size_limit:str,vault_fs_size:int,vault_fs_inode:int) -> tuple:
	'''
	This function gets the backup limits from a string.
	Use 0 for infinity, % numbers refer to disk usage %, use leading i to represent inodes, use comma to seperate multiple rules
	Bigger ones take precedence

	Parameters:
		backup_size_limit (str): The backup size limit string

	Returns:
		tuple: The backup size and inode limits	
	'''
	backup_size_limit = backup_size_limit.strip().lower()
	if not backup_size_limit or backup_size_limit == '0':
		return 0, 0
	backup_size_limits = backup_size_limit.split(',')
	rtn_size_limit = -1
	rtn_inode_limit = -1
	for limit in backup_size_limits:
		size_limit = -1
		inode_limit = -1
		if limit.endswith('%'):
			limit = limit.rstrip('%')
			if limit.startswith('i'):
				try:
					inode_limit = int(float(limit.lstrip('i')) * vault_fs_inode // 100)
					if vault_fs_inode and inode_limit == 0:
						# if vault has inodes but we cannot use any, set inode limit to 1
						inode_limit = 1
				except:
					pass
			else:
				try:
					size_limit = int(float(limit) * vault_fs_size // 100)
					if vault_fs_size and size_limit == 0:
						size_limit = 1
				except:
					pass
		elif limit.startswith('i'):
			try:
				inode_limit = int(limit.lstrip('i'))
			except:
				pass
		else:
			try:
				size_limit = int(limit)
			except:
				pass
		if size_limit > rtn_size_limit:
			rtn_size_limit = size_limit
		if inode_limit > rtn_inode_limit:
			rtn_inode_limit = inode_limit
	return rtn_size_limit, rtn_inode_limit
			


In [67]:
get_backup_limits_from_str('10%,i20%,2000',10000,100)

(2000, 20)

In [68]:
def get_path_fs_info(path:str):
	# this function gets the filesystem usage information of a path
	# df --no-sync --output=size,itotal <path>
	global DEBUG
	rtn = multiCMD.run_command(['df','--no-sync','--output=size,itotal',path],quiet=True)
	if rtn and rtn[-1]:
		try:
			size, inodes = rtn[-1].split(maxsplit=1)
			return int(size), int(inodes)
		except:
			pass
	return 0,0


In [69]:
rtn = multiCMD.run_command(['df','--no-sync','--output=size,itotal','.'],quiet=True)

In [70]:
get_path_fs_info('/tmp')

(31783256, 1048576)

In [71]:
latest_version_info = vault_info_dict[next(reversed(vault_info_dict))]
latest_version_info

VaultEntry(version_number=4, path='test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino', timestamp=1640995200.0, size=1153433, inode=4)

In [72]:
HASH_SIZE = 1<<24
HASH_SIZE

16777216

In [73]:
format_bytes(HASH_SIZE)

'16.00 Mi'

In [74]:
def hash_file(path,size = ...,full_hash=False):
	#From hpcp: https://github.com/yufei-pan/hpcp
	global HASH_SIZE
	if HASH_SIZE <= 0:
		# Do not hash
		return ''
	if size == ...:
		size = os.path.getsize(path)
	hasher = xxhash.xxh64()
	with open(path, 'rb') as f:
		if not full_hash:
			# Only hash the last hash_size bytes
			#f.seek(-min(1<<16,size), os.SEEK_END)
			f.seek(-min(HASH_SIZE,size), os.SEEK_END)
		for chunk in iter(lambda: f.read(4096), b''):
			hasher.update(chunk)
	return hasher.hexdigest()


In [75]:
{1} - {1} | {2} 

{2}

In [76]:
get_all_files_and_folders('test')

(['test/.X11-unix/X1024',
  'test/.X11-unix/X1025',
  'test/.X11-unix/X0',
  'test/.X11-unix/X1',
  'test/.ICE-unix/1549',
  'test/.ICE-unix/2599',
  'test/.X1024-lock',
  'test/.X1025-lock',
  'test/.X0-lock',
  'test/.X1-lock',
  'test/tt/test',
  'test/hello',
  'test/modify',
  'test/tmp/.X11-unix/X1024',
  'test/tmp/.X11-unix/X1025',
  'test/tmp/.X11-unix/X0',
  'test/tmp/.X11-unix/X1',
  'test/tmp/.ICE-unix/1549',
  'test/tmp/.ICE-unix/2599',
  'test/tmp/.ICE-unix/1539',
  'test/tmp/.ICE-unix/2571',
  'test/tmp/.X1024-lock',
  'test/tmp/.X1025-lock',
  'test/tmp/.X0-lock',
  'test/tmp/.X1-lock',
  'test/tmp/uv-2b31b12fc3a070b5.lock',
  'test/tmp/pyright-515312-HTP3Gm9Tt9sx/posixpath-515312-SIdxzh5eAA2W-.py',
  'test/tmp/pyright-515312-HTP3Gm9Tt9sx/builtins-515312-bXVHbkFb4HVW-.py',
  'test/tmp/inotify_test/test/testrecursivefolder/nestedFolder/nestedFile.txt',
  'test/tmp/inotify_test/dirMoveTest/subfolder.txt',
  'test/tmp/inotify_test/dirMoveTest/move_into_sub_folder.txt',
  't

In [77]:
get_all_files_and_folders('test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/')

(['test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.ICE-unix/1549',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.ICE-unix/2599',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1024',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X0',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1025',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/hi',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tt/test',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X1024',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X1025',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X0',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X1',
  'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.ICE-u

In [78]:
def check_duplicate(backupEntries:dict,latest_version_path:str,latest_version_files_folders:set,rel_entry:str,monitor_entry:str,isDir:bool):
	global DEBUG
	global tl
	if rel_entry in latest_version_files_folders:
		# this is a common file/folder
		latest_version_files_folders.remove(rel_entry)
		# check if the file is the same
		latest_version_entry = os.path.join(latest_version_path,rel_entry)
		if DEBUG:
			tl.teeprint(f'Checking {monitor_entry} against {latest_version_entry}')
		monitor_entry_stat = os.stat(monitor_entry)
		latest_version_entry_stat = os.stat(latest_version_entry)
		# do dev and inode based skip first
		try:
			if monitor_entry_stat.st_dev == latest_version_entry_stat.st_dev and monitor_entry_stat.st_ino == latest_version_entry_stat.st_ino:
				# same file
				if DEBUG:
					tl.teeprint(f'Same file according to dev and ino number {monitor_entry}')
				return
		except:
			if DEBUG:
				import traceback
				tl.teeerror(f'Error comparing dev and ino {monitor_entry}')
				tl.teeerror(traceback.format_exc())
		try:
			if monitor_entry_stat.st_size != latest_version_entry_stat.st_size:
				# size is different
				if DEBUG:
					tl.teeprint(f'Size different {monitor_entry}')
				backupEntries[monitor_entry] = BackupEntryValues(datetime.datetime.now().isoformat(),'modify',None)
			elif monitor_entry_stat.st_mtime_ns == latest_version_entry_stat.st_mtime_ns:
				# size and mtime the same, we assume as if the file content is the same
				if monitor_entry_stat.st_ctime_ns != latest_version_entry_stat.st_ctime_ns:
					# attrib changed
					if DEBUG:
						tl.teeprint(f'Size Same, mtime same, ctime different {monitor_entry}')
					backupEntries[monitor_entry] = BackupEntryValues(datetime.datetime.now().isoformat(),'attrib',None)
				elif DEBUG:
					tl.teeprint(f'Size Same, mtime same, ctime same {monitor_entry}')
			elif not isDir and hash_file(monitor_entry,monitor_entry_stat.st_size) != hash_file(latest_version_entry,latest_version_entry_stat.st_size):
				# size is same, but mtime is different, and hash is different
				if DEBUG:
					tl.teeprint(f'Size Same, mtime different, hash different {monitor_entry}')
				backupEntries[monitor_entry] = BackupEntryValues(datetime.datetime.now().isoformat(),'modify',None)
			elif isDir:
				# check for mode, uid, gid
				if monitor_entry_stat.st_mode != latest_version_entry_stat.st_mode or monitor_entry_stat.st_uid != latest_version_entry_stat.st_uid or monitor_entry_stat.st_gid != latest_version_entry_stat.st_gid:
					# attrib changed
					if DEBUG:
						tl.teeprint(f'Dir attrib changed {monitor_entry}')
					backupEntries[monitor_entry] = BackupEntryValues(datetime.datetime.now().isoformat(),'attrib',None)
				elif DEBUG:
					tl.teeprint(f'Dir attrib same {monitor_entry}')
			else:
				# size is same, but mtime is different, however hash is the same
				if DEBUG:
					tl.teeprint(f'Size Same, mtime different, hash same{monitor_entry}')
				backupEntries[monitor_entry] = BackupEntryValues(datetime.datetime.now().isoformat(),'attrib',None)
		except:
			if DEBUG:
				import traceback
				tl.teeerror(f'Error comparing size, mtime, hash {monitor_entry}')
				tl.teeerror(traceback.format_exc())
				tl.teeprint(f'Adding as modify {monitor_entry}')
			backupEntries[monitor_entry] = BackupEntryValues(datetime.datetime.now().isoformat(),'modify',None)
	else:
		# this is a new file/folder
		if DEBUG:
			tl.teeprint(f'New file/folder {monitor_entry}')
		backupEntries[monitor_entry] = BackupEntryValues(datetime.datetime.now().isoformat(),'create',None)

In [79]:
def delta_generate_backup_entries(backupEntries:dict,latest_version_info:VaultEntry,monitor_path:str):
	global DEBUG
	global tl
	# this function will generate the backup entries by comparing size, mtime and hash between latest version and the monitoring path.
	# When reverberator is in cold start, will not calculate hash to do space efficient move operations to vault.
	# When restarting reverberator, will do one backup that will only handle create, modify, delete, attrib.
	monitor_files, monitor_folders = get_all_files_and_folders(monitor_path)
	latest_version_files, latest_version_folders = get_all_files_and_folders(latest_version_info.path)
	latest_version_files_folders = set()
	for latest_version_entry in latest_version_files:
		latest_version_files_folders.add(os.path.relpath(latest_version_entry,latest_version_info.path))
	for latest_version_entry in latest_version_folders:
		latest_version_files_folders.add(os.path.relpath(latest_version_entry,latest_version_info.path)+'/')
	for monitor_entry in monitor_files:
		rel_entry = os.path.relpath(monitor_entry,monitor_path)
		isDir = False
		check_duplicate(backupEntries,latest_version_info.path,latest_version_files_folders,rel_entry,monitor_entry,isDir)
	for monitor_entry in monitor_folders:
		rel_entry = os.path.relpath(monitor_entry,monitor_path) + '/'
		isDir = True
		check_duplicate(backupEntries,latest_version_info.path,latest_version_files_folders,rel_entry,monitor_entry,isDir)
	for deleted_entry in latest_version_files_folders:
		# this is a deleted file/folder
		if DEBUG:
			tl.teeprint(f'Deleted file/folder {deleted_entry}')
		backupEntries[os.path.join(monitor_path,deleted_entry)] = BackupEntryValues(datetime.datetime.now().isoformat(),'delete',None)
	


In [80]:
backup_entries = OrderedDict()

In [81]:
latest_version_info

VaultEntry(version_number=4, path='test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino', timestamp=1640995200.0, size=1153433, inode=4)

In [82]:
delta_generate_backup_entries(backup_entries,latest_version_info,'test')
backup_entries

Checking test/.X11-unix/X1024 against test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1024
Same file according to dev and ino number test/.X11-unix/X1024
Checking test/.X11-unix/X1025 against test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1025
Same file according to dev and ino number test/.X11-unix/X1025
Checking test/.X11-unix/X0 against test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X0
Same file according to dev and ino number test/.X11-unix/X0
Checking test/.X11-unix/X1 against test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1
Same file according to dev and ino number test/.X11-unix/X1
Checking test/.ICE-unix/1549 against test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.ICE-unix/1549
Same file according to dev and ino number test/.ICE-unix/1549
Checking test/.ICE-unix/2599 against test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.ICE-unix/2599
Same file according to dev and ino n

OrderedDict([('test/test/',
              BackupEntryValues(iso_time='2025-03-28T14:57:49.972123', event='delete', source_path=None)),
             ('test/hi',
              BackupEntryValues(iso_time='2025-03-28T14:57:49.972155', event='delete', source_path=None))])

In [83]:
def get_all_folders(path):
	_, folders = get_all_files_and_folders(path)
	return folders

def get_all_files(path):
	files, _ = get_all_files_and_folders(path)
	return files

def get_all_files_and_folders(path):
	files = []
	folders = []
	try:
		with os.scandir(path) as entries:
			for entry in entries:
				if entry.is_dir(follow_symlinks=False):
					folders.append(entry.path+'/')
					child_files, child_folders = get_all_files_and_folders(entry.path)
					files.extend(child_files)
					folders.extend(child_folders)
				else:
					files.append(entry.path)
	except PermissionError:
		pass
	return files, folders


In [84]:
vault_info_dict

OrderedDict([(0,
              VaultEntry(version_number=0, path='test_vault/V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino', timestamp=1609488000.0, size=1395864371, inode=4200)),
             (1,
              VaultEntry(version_number=1, path='test_vault/V1--2023-01-01_00-00-00_-0800--100.0_MiB-1.0G_ino', timestamp=1672560000.0, size=104857600, inode=1000000000)),
             (2,
              VaultEntry(version_number=2, path='test_vault/V2--2020-01-01_00-00-00_-0800--1.0_MiB-1.00_ino', timestamp=1577865600.0, size=1048576, inode=1)),
             (3,
              VaultEntry(version_number=3, path='test_vault/V3--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino', timestamp=1609488000.0, size=1395864371, inode=4200)),
             (4,
              VaultEntry(version_number=4, path='test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino', timestamp=1640995200.0, size=1153433, inode=4))])

In [85]:
next(iter(vault_info_dict))

0

In [98]:
vault_info_dict

OrderedDict([(0,
              VaultEntry(version_number=0, path='test_vault/V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino', timestamp=1609488000.0, size=1395864371, inode=4200)),
             (1,
              VaultEntry(version_number=1, path='test_vault/V1--2023-01-01_00-00-00_-0800--100.0_MiB-1.0G_ino', timestamp=1672560000.0, size=104857600, inode=1000000000)),
             (2,
              VaultEntry(version_number=2, path='test_vault/V2--2020-01-01_00-00-00_-0800--1.0_MiB-1.00_ino', timestamp=1577865600.0, size=1048576, inode=1)),
             (3,
              VaultEntry(version_number=3, path='test_vault/V3--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino', timestamp=1609488000.0, size=1395864371, inode=4200)),
             (4,
              VaultEntry(version_number=4, path='test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino', timestamp=1640995200.0, size=1153433, inode=4))])

In [ ]:
#referenceVersionNumber, referenceVaultEntry = vault_info_dict.popitem(last=False)

In [ ]:
# referenceVaultPath = os.path.abspath(referenceVaultEntry.path)
# referenceVaultPath

'/var/home/kes/Documents/reverberator/test_vault/V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino'

In [93]:
get_all_files('test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino')

['test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.ICE-unix/1549',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.ICE-unix/2599',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1024',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X0',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/.X11-unix/X1025',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/hi',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tt/test',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X1024',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X1025',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X0',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.X11-unix/X1',
 'test_vault/V4--2022-01-01_00-00-00_+0000--1.1_MiB-4.0_ino/tmp/.ICE-unix/2599',
 '

In [129]:
def decrement_stepper(vault_info_dict:OrderedDict) -> tuple:
	# this function remove the oldest reverb from path
	if len(vault_info_dict) < 2:
		# we cannot step as there is less than 2 availble reverbs
		return 0 , 0
	referenceVersionNumber, referenceVaultEntry = vault_info_dict.popitem(last=False)
	referenceVaultPath = os.path.abspath(referenceVaultEntry.path)
	if referenceVaultPath == '/':
		print('Attempting to remove root, skipping')
		print(referenceVaultEntry)
		return 0 , 0
	if DEBUG:
		tl.teeprint(f'Removing {referenceVersionNumber}: {referenceVaultPath}')
	movedPath = {}
	for applyingVersionNumber in vault_info_dict:
		applyingVaultEntry = vault_info_dict[applyingVersionNumber]
		# now we need to go through all files in the applying vault entry
		# find all symlinks that is referencing an files in the reference vault entry
		# if it is in movedPath, we apply the new path to the symlink
		# if it is not in movedPath, this means it is first reference, thus
		#   we remove the symlink in the appying vault, move the file to the new location, and add the pair to movedPath
		path_files = get_all_files(applyingVaultEntry.path)
		if DEBUG:
			tl.teeprint(f"Dealing with {applyingVersionNumber}: {applyingVaultEntry.path} ({len(path_files)} files)")
		new_size = applyingVaultEntry.size
		pendingMovedPath = {}
		for file in path_files:
			if os.path.islink(file):
				target = os.path.realpath(file)
				if target in movedPath:
					# we just apply the new dest to the link
					new_target = os.path.relpath(movedPath[target],applyingVaultEntry.path)
					if DEBUG:
						tl.teeprint(f'Applying new target {new_target} to {file}')
					os.remove(file)
					os.symlink(new_target,file)
					continue
				if target.startswith(referenceVaultPath):
					# this means we need to move the file to the new location
					# get the relative path of the target
					if DEBUG:
						tl.teeprint(f'Moving {target} to {file}')
					os.remove(file)
					os.rename(target,file)
					new_size += os.path.getsize(file)
					pendingMovedPath[target] = os.path.relpath(file,applyingVaultEntry.path)
				elif DEBUG:
					tl.teeprint(f'Not moving {target} to {file}')
					tl.teeprint(f'{os.path.abspath(target)} not in {referenceVaultPath}')
		#V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino
		if new_size != applyingVaultEntry.size:
			backup_size_str = format_bytes(new_size,use_1024_bytes=True,to_str=True).replace(' ','_')
			newVaultPath = applyingVaultEntry.path.rpartition('--')[0] + f'--{backup_size_str}B-' + applyingVaultEntry.path.rpartition('-')[2]
			if newVaultPath != applyingVaultEntry.path:
				if DEBUG:
					tl.teeprint(f'Renaming {applyingVaultEntry.path} to {newVaultPath}')
				os.rename(applyingVaultEntry.path,newVaultPath)
				vault_info_dict[applyingVersionNumber] = applyingVaultEntry._replace(path=newVaultPath)
		else:
			newVaultPath = applyingVaultEntry.path
		for target in pendingMovedPath:
			movedPath[target] = os.path.join(newVaultPath,pendingMovedPath[target])
	# remove the reference vault entry
	if DEBUG:
		tl.teeprint(f'Removing {referenceVaultPath}')
	removingSize = get_path_size(referenceVaultPath)
	removingInodes = get_path_inodes(referenceVaultPath)
	multiCMD.run_command(['rm','-rf',referenceVaultPath],quiet=True,return_code_only=True)

	return removingSize, removingInodes

In [142]:
vault_info_dict, vault_size , vault_inodes= get_vault_info('test_vault')
decrement_stepper(vault_info_dict)

Removing 0: /var/home/kes/Documents/reverberator/test_vault/V0--2021-01-01_00-00-00_-0800--8.00_KiB-3.00_ino
Dealing with 1: test_vault/V1--2023-01-01_00-00-00_-0800--4.00_KiB-2.00_ino (1 files)
Not moving /var/home/kes/Documents/reverberator/test_vault/V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino/hello to test_vault/V1--2023-01-01_00-00-00_-0800--4.00_KiB-2.00_ino/hello
/var/home/kes/Documents/reverberator/test_vault/V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino/hello not in /var/home/kes/Documents/reverberator/test_vault/V0--2021-01-01_00-00-00_-0800--8.00_KiB-3.00_ino
Dealing with 2: test_vault/V2--2020-01-01_00-00-00_-0800--4.00_KiB-2.00_ino (1 files)
Not moving /var/home/kes/Documents/reverberator/test_vault/V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino/hello to test_vault/V2--2020-01-01_00-00-00_-0800--4.00_KiB-2.00_ino/hello
/var/home/kes/Documents/reverberator/test_vault/V0--2021-01-01_00-00-00_-0800--1.3_GiB-4.2K_ino/hello not in /var/home/kes/Documents/reverberator/tes

(8192, 3)

In [126]:
vault_info_dict

OrderedDict([(1,
              VaultEntry(version_number=1, path='test_vault/V1--2023-01-01_00-00-00_-0800--6.00_B-1000.00M_ino', timestamp=1672560000.0, size=6, inode=1000000000)),
             (2,
              VaultEntry(version_number=2, path='test_vault/V2--2020-01-01_00-00-00_-0800--1024.00_KiB-1.00_ino', timestamp=1577865600.0, size=1048576, inode=1)),
             (3,
              VaultEntry(version_number=3, path='test_vault/V3--2021-01-01_00-00-00_-0800--1.30_GiB-4.20K_ino', timestamp=1609488000.0, size=1395864371, inode=4200)),
             (4,
              VaultEntry(version_number=4, path='test_vault/V4--2022-01-01_00-00-00_+0000--1.10_MiB-4.00_ino', timestamp=1640995200.0, size=1153433, inode=4))])